# NbConvert

## Command line usage

`NbConvert` is both a library and command line tool that allows you to convert notebooks to other formats.  It ships with many common formats: `html`, `latex`, `markdown`, `python`, `rst`, and  `slides`
NbConvert relys on the Jinja templating engine, so implementing a new format or tweeking an existing one is easy.

You can invoke nbconvert by running

```bash
$ ipython nbconvert <options and arguments>
```

Call `ipython nbconvert` with the `--help` flag or without any aruments to display the basic help.  For detailed configuration help, use the `--help-all` flag.

### Basic export

As a test, the `Index.ipynb` notebook in the directory will be convert. 

If you're converting a notebook with code in it, make sure to run the code cells that you're interested in before attempting to convert the notebook.  Unless explicitly requested, nbconvert **does not execute the code cells** of the notebooks that it converts.

In [ ]:
%%bash
ipython nbconvert  'Index.ipynb'

Html is the (configurable) default value.  The verbose form of the same command as above is 

In [ ]:
%%bash
ipython nbconvert  --to=html 'Index.ipynb'

You can also convert to latex, which will extract the embeded images.  If the embeded images are SVGs, inkscape is used to convert them to pdf:

In [ ]:
%%bash
ipython nbconvert  --to=latex 'Index.ipynb'

Note that the latex conversion creates latex, not a PDF.  To create a PDF you need the required third party packages to compile the latex.

A `--post` flag is provided for convinience which allows you to have nbconvert automatically compile a PDF for you from your output.

In [ ]:
%%bash
ipython nbconvert  --to=latex 'Index.ipynb' --post=pdf

## Custom templates

Look at the first 20 lines of the `python` exporter

In [ ]:
pyfile = !ipython nbconvert --to python 'Index.ipynb' --stdout
for l in pyfile[20:40]:
    print l

From the code, you can see that non-code cells are also exported.  If you want to change this behavior, you can use a custom template.  The custom template inherits from the Python template and overwrites the markdown blocks so that they are empty.

In [ ]:
%%writefile simplepython.tpl
{% extends 'python.tpl'%}

{% block markdowncell -%}
{% endblock markdowncell %}

## we also want to get rig of header cell
{% block headingcell -%}
{% endblock headingcell %}

## and let's change the appearance of input prompt
{% block in_prompt %}
# This was input cell with prompt number : {{ cell.prompt_number if cell.prompt_number else ' ' }}
{%- endblock in_prompt %}

In [ ]:
pyfile = !ipython nbconvert --to python 'Index.ipynb' --stdout --template=simplepython.tpl

for l in pyfile[4:40]:
    print l
print '...'

For details about the template syntax, refer to [Jinja's manual](http://jinja2.readthedocs.org/en/latest/intro.html).

## Template that use cells metadata

The notebook file format supports attaching arbitrary JSON metadata to each cell.  Here, as an exercise, you will use the metadata to tags cells.

First you need to choose another notebook you want to convert to html, and tag some of the cells with metadata.  You can refere to the file `soln/celldiff.js` as an example or follow the Javascript tutorial to figure out how do change cell metadata.  Assuming you have a notebook with some of the cells tagged as `Easy`|`Medium`|`Hard`|`<None>`, the notebook can be converted specially using a custom template.  Design your template in the cells provided below.

The following, unorganized lines of code, may be of help:

```
{% extends 'html_full.tpl'%}
{% block any_cell %}
{{ super() }}
<div style="background-color:red">
<div style='background-color:orange'>
```

If your key name under `cell.metadata.example.difficulty`, the following code would get the value of it:

`cell['metadata'].get('example',{}).get('difficulty','')`

Tip: Use `%%writefile` to edit the template in the notebook.

In [ ]:
%%bash
# ipython nbconvert --to html <your chosen notebook.ipynb> --template=<your template file>

In [ ]:
%loadpy soln/coloreddiff.tpl

In [ ]:
# ipython nbconvert --to html '04 - Custom Display Logic.ipynb' --template=soln/coloreddiff.tpl

### Get rid of all command line flags.

IPython nbconvert can be configured using the default profile or a profile specified via the `--profile` flag.  Additionally, if a `config.py` file exist in current working directory, nbconvert will use that as config.